经验证：
* 1、test的时间顺序并不是全部都是时序的，排行榜是按时序排列后取前0.78为public，后0.22为private，所以以后的工作重心将落到train的后44%
* 2、测试集中私有部分从2018-07-24 09:00:00开始，有9173472条数据，包含全部的building_id
* 3、测试集中公共部分与私有部分对应的数据从2017-07-24 09:00:00开始，有9174900条数据，包含全部的building_id
* 4、重点为训练集中2017-07-24 09:00:00后的数据建模，有9003109条数据，占训练集0.445
* 5、为防止测试集中不评分的那部分不知道算不算在0.22里面，也为防止一定的过拟合，可以考虑重点关注2017-07月后的训练数据，占0.51
* 6、验证EDA1后，发现0-104号建筑5月20日前的大量为0的数据是有误的，训练集不应该对这部分数据进行过拟合，删除这部分数据训练对提高public评分很有帮助，遗憾的是，private并不包含这类数据
* 7、猜想某些大范围为0的数据都同0-104号一样是有误的，重点验证7月后的该种数据，测试是否能提高public

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)


py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/ashrae-energy-prediction.zip
../input/ashrae-energy-prediction/test.csv


In [2]:
# 导入数据, 只有train 和 test
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')

In [3]:
# 时间戳转换日期
train["timestamp"] = pd.to_datetime(train["timestamp"])
test["timestamp"] = pd.to_datetime(test["timestamp"])

In [4]:
train_exception = pd.read_pickle('../output/ashrae-eda-exception-label1/train_exception.pkl')
train['exception'] = train_exception.exception.values
del train_exception
gc.collect()

31

In [5]:
test_exception = pd.read_pickle('../output/ashrae-eda-exception-label1/test_exception.pkl')
test['exception'] = test_exception.exception.values
del test_exception
gc.collect()

20

In [6]:
gc.collect()

20

In [7]:
# 1413个建筑有0，498个建筑有1，324个建筑有2，145个建筑有3
# [0,1]有132个，[0,2]64，[0,3]21, [1,2]13
# [0,1,2]220, [0,1,3]111
# [0,1,2,3]13

In [8]:
gc.collect()

40

In [9]:
# (0,0)到(104,0)从5月中旬才开始有读数, 其中有些在开始有读数之前有一些读数，不知什么情况
# 猜测该地区出现了大面积的电表故障或者大面积的电力故障，其中有极少建筑幸存或用于检修测试，所以有少量读数，5月中旬后修好才恢复该地区正常用电或读表

In [10]:
train.exception.value_counts(dropna=False)

 0    18065316
-1     1670842
 1      314805
-2       85465
 2       79672
Name: exception, dtype: int64

In [11]:
test.exception.value_counts(dropna=False)

 0.0    36041924
-1.0     3388094
 NaN     1281846
 1.0      656374
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [12]:
def exception_label(bid, meter, start, end):
    print(f'标记bid={bid}, meter={meter}异常')
    train.loc[(train.building_id == bid)  & (train.meter == meter) & (train.timestamp >= start) & (train.timestamp <= end), 'exception'] = 1
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2017')) & (test.timestamp <= end.replace('2016', '2017')), 'exception'] = 1
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2018')) & (test.timestamp <= end.replace('2016', '2018')), 'exception'] = 1
    gc.collect()

In [13]:
exception_label(745, 2, '2016-10-04 08:00:00', '2016-10-11 18:00:00')

# exception_label(747, 2, '2016-05-23 08:00:00', '2016-06-08 12:00:00')
# exception_label(747, 2, '2016-06-11 06:00:00', '2016-06-18 03:00:00')
# exception_label(747, 2, '2016-06-18 07:00:00', '2016-07-02 03:00:00')
# exception_label(747, 2, '2016-07-02 05:00:00', '2016-09-16 01:00:00')
# exception_label(747, 2, '2016-09-17 05:00:00', '2016-09-25 01:00:00')
# exception_label(747, 2, '2016-10-18 16:00:00', '2016-10-22 01:00:00')

exception_label(750, 2, '2016-10-04 08:00:00', '2016-10-11 08:00:00')

exception_label(753, 2, '2016-10-04 08:00:00', '2016-10-11 09:00:00')

# exception_label(754, 2, '2016-06-02 19:00:00', '2016-06-08 03:00:00')
# exception_label(754, 2, '2016-06-15 10:00:00', '2016-07-02 03:00:00')
# exception_label(754, 2, '2016-07-02 05:00:00', '2016-10-04 09:00:00')

# exception_label(757, 2, '2016-06-11 11:00:00', '2016-07-02 03:00:00')
# exception_label(757, 2, '2016-07-02 05:00:00', '2016-10-11 08:00:00')

exception_label(758, 2, '2016-03-07 13:00:00', '2016-03-13 02:00:00')
exception_label(758, 2, '2016-03-13 04:00:00', '2016-04-22 04:00:00')
exception_label(758, 2, '2016-04-22 06:00:00', '2016-07-02 03:00:00')
exception_label(758, 2, '2016-07-02 05:00:00', '2016-10-11 08:00:00')
exception_label(758, 2, '2016-10-11 10:00:00', '2016-12-29 05:00:00')
exception_label(758, 2, '2016-12-29 07:00:00', '2016-12-31 23:00:00')

# exception_label(759, 2, '2016-01-14 22:00:00', '2016-01-25 08:00:00')
# exception_label(759, 2, '2016-01-25 11:00:00', '2016-02-02 08:00:00')
# exception_label(759, 2, '2016-06-12 12:00:00', '2016-06-24 13:00:00')
# exception_label(759, 2, '2016-07-26 18:00:00', '2016-08-05 12:00:00')
# exception_label(759, 2, '2016-08-05 14:00:00', '2016-09-06 19:00:00')

exception_label(762, 2, '2016-02-23 12:00:00', '2016-03-03 15:00:00')
exception_label(762, 2, '2016-03-03 17:00:00', '2016-03-13 02:00:00')
exception_label(762, 2, '2016-03-13 04:00:00', '2016-04-22 04:00:00')
exception_label(762, 2, '2016-04-22 06:00:00', '2016-07-02 03:00:00')
exception_label(762, 2, '2016-07-02 05:00:00', '2016-10-11 08:00:00')
exception_label(762, 2, '2016-10-11 10:00:00', '2016-12-29 05:00:00')
exception_label(762, 2, '2016-12-29 07:00:00', '2016-12-31 23:00:00')

exception_label(767, 2, '2016-10-04 08:00:00', '2016-10-11 08:00:00')

# exception_label(769, 2, '2016-07-03 10:00:00', '2016-09-02 08:00:00')
# exception_label(769, 2, '2016-10-04 05:00:00', '2016-10-11 08:00:00')

# exception_label(771, 2, '2016-05-23 10:00:00', '2016-07-02 03:00:00')
# exception_label(771, 2, '2016-07-02 05:00:00', '2016-10-11 08:00:00')
# exception_label(771, 2, '2016-10-16 12:00:00', '2016-10-22 01:00:00')

exception_label(772, 2, '2016-01-01 00:00:00', '2016-02-09 07:00:00')
exception_label(772, 2, '2016-02-09 09:00:00', '2016-02-25 14:00:00')
exception_label(772, 2, '2016-10-04 08:00:00', '2016-10-18 14:00:00')
exception_label(772, 2, '2016-10-18 16:00:00', '2016-12-16 12:00:00')

# exception_label(774, 2, '2016-01-01 00:00:00', '2016-02-11 08:00:00')
# exception_label(774, 2, '2016-07-29 08:00:00', '2016-08-04 06:00:00')
# exception_label(774, 2, '2016-08-04 09:00:00', '2016-08-22 22:00:00')
# exception_label(774, 2, '2016-08-23 08:00:00', '2016-08-29 20:00:00')

exception_label(776, 2, '2016-08-25 09:00:00', '2016-08-29 10:00:00')
exception_label(776, 2, '2016-10-18 00:00:00', '2016-10-26 11:00:00')
exception_label(776, 2, '2016-10-26 16:00:00', '2016-11-18 09:00:00')
exception_label(776, 2, '2016-11-20 16:00:00', '2016-11-25 23:00:00')
exception_label(776, 2, '2016-11-29 16:00:00', '2016-12-09 05:00:00')

exception_label(783, 2, '2016-01-01 00:00:00', '2016-12-09 14:00:00')

exception_label(784, 2, '2016-10-04 08:00:00', '2016-10-11 08:00:00')

# exception_label(789, 2, '2016-06-24 07:00:00', '2016-07-13 05:00:00')
# exception_label(789, 2, '2016-07-20 13:00:00', '2016-08-07 12:00:00')

# exception_label(790, 2, '2016-06-24 07:00:00', '2016-07-19 13:00:00')
# exception_label(790, 2, '2016-07-19 15:00:00', '2016-08-07 12:00:00')
# exception_label(790, 2, '2016-08-07 14:00:00', '2016-09-15 11:00:00')
# exception_label(790, 2, '2016-09-15 14:00:00', '2016-09-29 11:00:00')

# exception_label(791, 2, '2016-05-26 04:00:00', '2016-05-31 18:00:00')
# exception_label(791, 2, '2016-06-01 04:00:00', '2016-06-05 09:00:00')
# exception_label(791, 2, '2016-06-05 11:00:00', '2016-10-12 08:00:00')

# exception_label(796, 2, '2016-05-24 06:00:00', '2016-10-24 12:00:00')

# exception_label(798, 2, '2016-05-24 10:00:00', '2016-10-18 08:00:00')

# exception_label(799, 2, '2016-05-13 14:00:00', '2016-05-20 11:00:00')
# exception_label(799, 2, '2016-05-20 13:00:00', '2016-07-05 18:00:00')
# exception_label(799, 2, '2016-07-05 20:00:00', '2016-10-24 15:00:00')
# exception_label(799, 2, '2016-10-28 19:00:00', '2016-11-03 13:00:00')

# exception_label(802, 2, '2016-05-24 06:00:00', '2016-06-05 13:00:00')
# exception_label(802, 2, '2016-06-05 15:00:00', '2016-07-24 11:00:00')
# exception_label(802, 2, '2016-07-24 15:00:00', '2016-10-11 05:00:00')

exception_label(907, 2, '2016-09-30 16:00:00', '2016-10-08 03:00:00') # 中国国庆节？

exception_label(912, 2, '2016-10-23 12:00:00', '2016-10-27 10:00:00')
exception_label(912, 2, '2016-10-27 12:00:00', '2016-11-04 05:00:00')

exception_label(928, 2, '2016-10-08 17:00:00', '2016-10-18 09:00:00')
exception_label(928, 2, '2016-10-23 16:00:00', '2016-10-27 08:00:00')
exception_label(928, 2, '2016-10-27 17:00:00', '2016-10-31 08:00:00')

exception_label(954, 2, '2016-01-01 00:00:00', '2016-08-08 10:00:00')

exception_label(1072, 2, '2016-01-01 00:00:00', '2016-07-25 12:00:00')

# exception_label(1075, 2, '2016-05-24 10:00:00', '2016-10-25 08:00:00')

# exception_label(1078, 2, '2016-05-03 15:00:00', '2016-05-10 10:00:00')
# exception_label(1078, 2, '2016-05-10 12:00:00', '2016-07-15 08:00:00')
# exception_label(1078, 2, '2016-07-15 10:00:00', '2016-07-28 15:00:00')
# exception_label(1078, 2, '2016-07-28 21:00:00', '2016-09-28 14:00:00')
# exception_label(1078, 2, '2016-09-28 16:00:00', '2016-10-01 12:00:00')
# exception_label(1078, 2, '2016-10-01 14:00:00', '2016-10-04 09:00:00')
# exception_label(1078, 2, '2016-10-04 11:00:00', '2016-10-11 10:00:00')

# 1099问题太奇怪了

exception_label(1111, 2, '2016-09-19 10:00:00', '2016-09-27 15:00:00')
exception_label(1111, 2, '2016-09-28 02:00:00', '2016-10-11 20:00:00')

# exception_label(1119, 2, '2016-06-09 21:00:00', '2016-06-15 14:00:00')
# exception_label(1119, 2, '2016-06-15 16:00:00', '2016-07-01 18:00:00')
# exception_label(1119, 2, '2016-07-01 20:00:00', '2016-07-28 16:00:00')
# exception_label(1119, 2, '2016-07-28 21:00:00', '2016-08-31 09:00:00')
# exception_label(1119, 2, '2016-08-31 11:00:00', '2016-09-08 15:00:00')

# exception_label(1149, 2, '2016-04-12 11:00:00', '2016-10-26 17:00:00')

# exception_label(1155, 2, '2016-05-25 10:00:00', '2016-08-25 07:00:00')
# exception_label(1155, 2, '2016-08-25 09:00:00', '2016-09-22 08:00:00')
# exception_label(1155, 2, '2016-09-22 10:00:00', '2016-10-18 09:00:00')

# exception_label(1162, 2, '2016-05-24 10:00:00', '2016-07-18 09:00:00')
# exception_label(1162, 2, '2016-07-18 11:00:00', '2016-09-11 02:00:00')
# exception_label(1162, 2, '2016-09-11 04:00:00', '2016-09-17 19:00:00')
# exception_label(1162, 2, '2016-09-20 06:00:00', '2016-09-29 16:00:00')

# exception_label(1186, 2, '2016-07-05 09:00:00', '2016-08-05 13:00:00')

# exception_label(1192, 2, '2016-07-07 20:00:00', '2016-07-19 14:00:00')

# 1203问题奇怪

# 1207也奇怪

# 1220小缺口奇怪

exception_label(1225, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1226, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1243, 2, '2016-10-19 15:00:00', '2016-11-08 20:00:00')

exception_label(1250, 2, '2016-01-01 00:00:00', '2016-12-21 15:00:00')

exception_label(1263, 2, '2016-11-10 14:00:00', '2016-11-21 14:00:00')

exception_label(1288, 2, '2016-07-07 16:00:00', '2016-08-10 18:00:00')

exception_label(1291, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1292, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1293, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1294, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1295, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1296, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1297, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1298, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1299, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1305, 2, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1307, 2, '2016-09-28 07:00:00', '2016-10-20 12:00:00')
exception_label(1307, 2, '2016-12-17 06:00:00', '2016-12-31 04:00:00') # 处理了1.0613的值

exception_label(1309, 2, '2016-07-25 20:00:00', '2016-08-04 11:00:00')

exception_label(1329, 2, '2016-11-21 16:00:00', '2016-11-29 09:00:00')

exception_label(1346, 2, '2016-12-01 08:00:00', '2016-12-06 17:00:00')

exception_label(1361, 2, '2016-11-28 12:00:00', '2016-12-01 10:00:00')
exception_label(1361, 2, '2016-12-01 13:00:00', '2016-12-06 15:00:00')

标记bid=745, meter=2异常
标记bid=750, meter=2异常
标记bid=753, meter=2异常
标记bid=758, meter=2异常
标记bid=758, meter=2异常
标记bid=758, meter=2异常
标记bid=758, meter=2异常
标记bid=758, meter=2异常
标记bid=758, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=762, meter=2异常
标记bid=767, meter=2异常
标记bid=772, meter=2异常
标记bid=772, meter=2异常
标记bid=772, meter=2异常
标记bid=772, meter=2异常
标记bid=776, meter=2异常
标记bid=776, meter=2异常
标记bid=776, meter=2异常
标记bid=776, meter=2异常
标记bid=776, meter=2异常
标记bid=783, meter=2异常
标记bid=784, meter=2异常
标记bid=907, meter=2异常
标记bid=912, meter=2异常
标记bid=912, meter=2异常
标记bid=928, meter=2异常
标记bid=928, meter=2异常
标记bid=928, meter=2异常
标记bid=954, meter=2异常
标记bid=1072, meter=2异常
标记bid=1111, meter=2异常
标记bid=1111, meter=2异常
标记bid=1225, meter=2异常
标记bid=1226, meter=2异常
标记bid=1243, meter=2异常
标记bid=1250, meter=2异常
标记bid=1263, meter=2异常
标记bid=1288, meter=2异常
标记bid=1291, meter=2异常
标记bid=1292, meter=2异常
标记bid=1293, meter=2异常
标

In [14]:
gc.collect()

20

In [15]:
%%time
for bid in train[(train.exception == 1) & (train.meter == 2)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 2) & (train.exception == 0), 'exception'] = -1
    test.loc[(test.building_id == bid) & (test.meter == 2) & (test.exception != 1), 'exception'] = -1

CPU times: user 17.8 s, sys: 13.5 s, total: 31.3 s
Wall time: 31.3 s


In [16]:
train.exception.value_counts(dropna=False)

 0    17761403
-1     1924531
 1      365029
-2       85465
 2       79672
Name: exception, dtype: int64

In [17]:
test.exception.value_counts(dropna=False)

 0.0    35435620
-1.0     3920902
 NaN     1239910
 1.0      771806
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [18]:
# ## 存疑异常标记

# # 30前期有大量0值
# train.loc[(train.building_id == 30) & (train.meter == 1) & (train.meter_reading < 212), 'exception'] = 2

# # 43大量0值
# train.loc[(train.building_id == 43) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

# # 28大量0值
# train.loc[(train.building_id == 28) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

In [19]:
gc.collect()

80

In [20]:
train[['exception']].to_pickle('../output/fork-of-ashrae-eda-exception-label2/train_exception.pkl')
test[['exception']].to_pickle('../output/fork-of-ashrae-eda-exception-label2/test_exception.pkl')